<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Statsmodels Lab

---

In [1]:
import warnings
warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(font_scale=1.5)

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

## Load the data

In [3]:
df = pd.read_csv('../../../../resource-datasets/auto_stats/Auto.csv')

## Do any necessary cleaning steps

In [14]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [16]:
df['horsepower'] = df.apply(lambda x: np.nan if x['horsepower']=='?' else x['horsepower'], axis=1)

In [18]:
df.dropna(inplace=True)

In [29]:
df['horsepower'] = df.horsepower.astype('int')
df.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower        int64
weight            int64
acceleration    float64
year              int64
origin            int64
name             object
dtype: object

## Use statsmodels to fit a linear regression model predicting `mpg`

### Use data frames

Describe what the model tells about 
- confidence intervals
- model quality
- residuals

In [32]:
X = df.loc[:,'cylinders':'year']
formula = 'mpg ~ ' + ('+').join(X.columns)
y, X = patsy.dmatrices(formula, data=df, return_type='dataframe')

In [33]:
# remember to use the lowercase ols in order to support patsy formulas:

model = smf.ols(formula=formula, data=df)
results = model.fit()
results.summary()

# Adj R2 is very good.
# Very low P(F-statistic) which means we can reject the null hypothesis that we set all coefficients to 0, ie.
# we're better off with this model than with baseline. Also a high log-likelihood, but this is a relative number
# which we need to compare vs. other models.
# CI is very wide for the intercept, also there are 3 variables that could be removed?
# Residuals are positively-skewed

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.809
Model:                            OLS   Adj. R-squared:                  0.806
Method:                 Least Squares   F-statistic:                     272.2
Date:                Wed, 05 Feb 2020   Prob (F-statistic):          3.79e-135
Time:                        14:04:35   Log-Likelihood:                -1036.5
No. Observations:                 392   AIC:                             2087.
Df Residuals:                     385   BIC:                             2115.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -14.5353      4.764     -3.051      0.002     -23.902      -5.169
cylinders       -0.3299      0.332     -0.993      0.321      -0.983       0.323
displacement     0.0077      0.007      1.044      0.297      -0.007       0.022
horsepower      -0.0004      0.014     -0.028      0.977      -0.028       0.027
weight          -0.0068      0.001    -10.141      0.000      -0.008      -0.005
acceleration     0.0853      0.102      0.836      0.404      -0.115       0.286
year             0.7534      0.053     14.318      0.000       0.650       0.857
==============================================================================
Omnibus:                       37.865   Durbin-Watson:                   1.232
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               60.248
Skew:                           0.630   Prob(JB):                     8.26e-14
Kurtosis:                       4.449   Cond. No.                     8.53e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.53e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Use the formula language

Experiment with the formula language. Include interaction terms or nonlinear predictor dependence.

**Hint:** If you want to use patsy with categorical variables, you can dummify within the formula
with 

`formula = 'mpg~C(origin)'`

or

`formula = 'mpg~C(origin, Treatment(reference=2))'`

where the latter allows you to decide which category to use as the reference point rather than just dropping first.

In [47]:
X = df.loc[:,'cylinders':'year']
formula = 'mpg ~ ' + ('+').join(X.columns) + '+ C(origin, Treatment(reference=1))'
y, X = patsy.dmatrices(formula, data=df, return_type='dataframe')

model = smf.ols(formula=formula, data=df)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.824
Model:                            OLS   Adj. R-squared:                  0.821
Method:                 Least Squares   F-statistic:                     224.5
Date:                Wed, 05 Feb 2020   Prob (F-statistic):          1.79e-139
Time:                        14:23:46   Log-Likelihood:                -1020.5
No. Observations:                 392   AIC:                             2059.
Df Residuals:                     383   BIC:                             2095.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                -17.9546      4.677     -3.839      0.000     -27.150      -8.759
C(origin, Treatment(reference=1))[T.2]     2.6300      0.566      4.643      0.000       1.516       3.744
C(origin, Treatment(reference=1))[T.3]     2.8532      0.553      5.162      0.000       1.766       3.940
cylinders                                 -0.4897      0.321     -1.524      0.128      -1.121       0.142
displacement                               0.0240      0.008      3.133      0.002       0.009       0.039
horsepower                                -0.0182      0.014     -1.326      0.185      -0.045       0.009
weight                                    -0.0067      0.001    -10.243      0.000      -0.008      -0.005
acceleration                               0.0791      0.098      0.805      0.421      -0.114       0.272
year                                       0.7770      0.052     15.005      0.000       0.675       0.879
==============================================================================
Omnibus:                       23.395   Durbin-Watson:                   1.291
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               34.452
Skew:                           0.444   Prob(JB):                     3.30e-08
Kurtosis:                       4.150   Cond. No.                     8.70e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.7e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Fit with regularization

In [79]:
results_reg = model.fit_regularized(alpha=1, L1_wt=0.5, refit=False)
results_reg.params

Intercept                                 0.000000
C(origin, Treatment(reference=1))[T.2]    0.000000
C(origin, Treatment(reference=1))[T.3]    0.000000
cylinders                                 0.000000
displacement                             -0.051363
horsepower                                0.045152
weight                                   -0.001530
acceleration                              0.880540
year                                      0.258433
dtype: float64

## Bonus: Construct an sklearn wrapper for statsmodels

Follow the example [here](http://nelsonauner.com/data/2018/05/21/wrap-statsmodels-in-sklearn.html) and use cross validation.

In [51]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [52]:
class SMFormulaWrapper(BaseEstimator, RegressorMixin):
    """ A sklearn-style wrapper for formula-based statsmodels regressors """
    def __init__(self, model_class, formula):
        self.model_class = model_class
        self.formula = formula
    def fit(self, X, y=None):
        self.model_ = self.model_class(self.formula, data=X)
        self.results_ = self.model_.fit()
    def predict(self, X):
        return self.results_.predict(X)

In [81]:
df_sub = df.loc[:,'cylinders':'year']
formula = 'mpg ~ ' + ('+').join(df_sub.columns) + ' + C(origin, Treatment(reference=1))'
# you could use a for loop where you sequentially append onto formula if you want include the 2nd power of terms

cross_val_score(SMFormulaWrapper(smf.ols, formula), df, y, cv=5).mean()

0.5936618440509804